In [ ]:
!pip install scikeras

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from scikeras.wrappers import KerasClassifier
# from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt

In [ ]:
data = load_iris()
X = data.data
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def create_model(learning_rate, num_units=64):
  model = keras.Sequential([
    keras.layers.Dense(units=num_units, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(units=num_units, activation='relu'),
    keras.layers.Dense(units=3, activation='softmax') # Multi-class classification
  ])
  optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
  model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
model = KerasClassifier(build_fn=create_model, epochs=5, batch_size=10, learning_rate = [0.001, 0.01, 0.1], num_units = [32, 64, 128])

param_grid = { 'learning_rate' : [0.001, 0.01, 0.1], 'num_units' : [32, 64, 128] }
grid_search = GridSearchCV(estimator=model, param_grid = param_grid, cv=3, verbose=1)
grid_result = grid_search.fit(X_train, y_train)

print(f"Best Parameters: {grid_result.best_params_}")
print(f"Best Accuracy: {grid_result.best_score_}")

best_model = grid_result.best_estimator_
history = best_model.fit(X_train, y_train, epochs=30, batch_size=32)

# test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {test_accuracy}")